In [1]:
# Load AIEEE Data

In [3]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [4]:
import sys  
sys.path.insert(0, 'Models/ERData/PreProcessing/')

from er_preprocess import *

In [5]:
er_df = ERData_MajorityLabel()

Load Models

In [6]:
#  other models
AIEEE_BERT = "Models/AIEEEGenderData/BERT/SavedModel/BERT"
CBSE_BERT = "Models/CBSEData/BERT/SavedModel/BERT"
ER_BERT = "Models/ERData/BERT/SavedModel/BERT"

In [7]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [8]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [9]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_BERT)

In [10]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_BERT)

In [11]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_BERT)

In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


Predict Gender

In [28]:
from transformers import pipeline
from sklearn.metrics import classification_report
import math
import torch

In [35]:
pipe = pipeline(task="text-classification",model=cbse_model, tokenizer=tokenizer)

In [36]:
pipe(list(er_df[:10]['Name']))

[{'label': 'LABEL_0', 'score': 0.9981642365455627},
 {'label': 'LABEL_1', 'score': 0.9946187734603882},
 {'label': 'LABEL_0', 'score': 0.815380871295929},
 {'label': 'LABEL_1', 'score': 0.9991344809532166},
 {'label': 'LABEL_0', 'score': 0.9993513226509094},
 {'label': 'LABEL_1', 'score': 0.9927660226821899},
 {'label': 'LABEL_1', 'score': 0.9820523858070374},
 {'label': 'LABEL_0', 'score': 0.9953339695930481},
 {'label': 'LABEL_1', 'score': 0.9992725253105164},
 {'label': 'LABEL_0', 'score': 0.9954609274864197}]

In [37]:
tokens = tokenizer(list(er_df['Name']), padding=True, truncation=True, return_tensors="pt")
output = rmodel(**tokens)

AttributeError: 'TextClassificationPipeline' object has no attribute 'postprocess'

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [44]:
def print_results_old(rmodel, tokenizer, df):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    report = classification_report(true, labels, digits = 4)
    print(report)
    return labels

In [16]:
er_df.shape

(7149489, 2)

In [41]:
def print_results(name):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    
    aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_BERT)
    cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_BERT)
    
    # Extract tokenizer.
    tokens = tokenizer(name, padding=True, truncation=True, return_tensors="pt")
    output = aieee_model(**tokens)
    predictions = torch.nn.functional.softmax(output.logits, dim=-1)
    aieee_label = torch.argmax(predictions)
    
    output = cbse_model(**tokens)
    predictions = torch.nn.functional.softmax(output.logits, dim=-1)
    cbse_label = torch.argmax(predictions)
    
    
    return cbse_label, aieee_label


In [45]:
report = print_results_old(cbse_model, tokenizer, er_df[:10])

              precision    recall  f1-score   support

         0.0     0.8000    0.8000    0.8000         5
         1.0     0.8000    0.8000    0.8000         5

    accuracy                         0.8000        10
   macro avg     0.8000    0.8000    0.8000        10
weighted avg     0.8000    0.8000    0.8000        10



In [46]:
f __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, list(er_df['Name'])))

[0, 1, 0, 1, 0, 1, 1, 0, 1, 0]

In [47]:
op = print_results(cbse_model, tokenizer, er_df[:1000])

tensor([0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
        1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
        0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1,
        1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
        0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0,

In [ ]:
print_results(aieee_model, tokenizer, er_df)

In [49]:
er_df.shape

(7149489, 2)